<a href="https://colab.research.google.com/github/akash166d/RAG_LLM_Template/blob/main/HomeMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Env

In [ ]:
!pip install pandas
!pip install langchain==0.0.316
!pip install openai==0.28.1
!pip install chromadb

!pip install numpy
# !pip install -U langchain-openai
!pip install pydantic
!pip install shutil



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain-0.0.316-py3-none-any.whl (1.9 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.12
    Uninstalling langchain-0.2.12:
      Successfully uninstalled langchain-0.2.12
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-0.28.1-py3-none-any.whl (76 kB)


  Attempting uninstall: openai
    Found existing installation: openai 1.39.0
    Uninstalling openai-1.39.0:
      Successfully uninstalled openai-1.39.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.20 requires openai<2.0.0,>=1.32.0, but you have openai 0.28.1 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [ ]:
import os
import pandas as pd
import shutil
from dataclasses import dataclass

from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.evaluation import load_evaluator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores.chroma import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, NonNegativeInt
from langchain.prompts import PromptTemplate
from fastapi.encoders import jsonable_encoder

# Rubrik1 - Synthetic Data Generation

Define LLM

In [ ]:


os.environ["OPENAI_API_KEY"] = "voc-xxxxxxxxxxxxxxxxxxxxxxxxxxx"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2500)



/home/student/.local/lib/python3.10/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/student/.local/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Prompt

In [ ]:

INSTRUCTION = "Generate a CSV file with at least 10 real estate listings."
SAMPLE_LISTING = \
"""
Neighborhood: Streterville
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 3,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Streterville. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Streterville gem.

Neighborhood Description: Streterville is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Streterville Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

In [ ]:
class RealEstateListing(BaseModel):
    """
    A real estate listing.

    Attributes:
    - neighborhood: str
    - price: NonNegativeInt
    - bedrooms: NonNegativeInt
    - bathrooms: NonNegativeInt
    - house_size: NonNegativeInt
    - description: str
    - neighborhood_description: str
    """
    neighborhood: str = Field(description="The neighborhood where the property is located")
    price: NonNegativeInt = Field(description="The price of the property in USD")
    bedrooms: NonNegativeInt = Field(description="The number of bedrooms in the property")
    bathrooms: NonNegativeInt = Field(description="The number of bathrooms in the property")
    house_size: NonNegativeInt = Field(description="The size of the house in square feet")
    description: str = Field(description="A description of the property")
    neighborhood_description: str = Field(description="A description of the neighborhood.")

class ListingCollection(BaseModel):
    """
    A collection of real estate listings.

    Attributes:
    - listings: List[RealEstateListing]
    """
    listings: List[RealEstateListing] = Field(description="A list of real estate listings")

In [ ]:
# generate parsed output
parser = PydanticOutputParser(pydantic_object=ListingCollection)

In [ ]:
# printing the prompt
prompt = PromptTemplate(
    template="{instruction}\n{sample}\n{format_instructions}\n",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

query = prompt.format(
    instruction=INSTRUCTION,
    sample=SAMPLE_LISTING,
)
print(type(query))

<class 'str'>


## Rubrik 1.1 : Generating Real Estate Listings with an LLM

In [ ]:
# get the response
response = llm(query)

In [ ]:
# create a dataframe from the response
result = parser.parse(response)
df = pd.DataFrame(jsonable_encoder(result.listings))
df.head(10)

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Streterville,800000,3,2,3000,Welcome to this eco-friendly oasis nestled in ...,"Streterville is a close-knit, environmentally-..."
1,Downtown,1200000,4,3,4000,Luxury living in the heart of Downtown. This s...,Downtown is a bustling urban neighborhood with...
2,Lakeview,650000,2,2,2500,"Charming 2-bedroom, 2-bathroom home in the des...",Lakeview is a family-friendly neighborhood wit...
3,Lincoln Park,950000,3,3,3200,Modern elegance in the heart of Lincoln Park. ...,Lincoln Park is a vibrant neighborhood known f...
4,West Loop,1100000,3,2,2800,"Sleek and stylish 3-bedroom, 2-bathroom loft i...",West Loop is a hip and happening neighborhood ...
5,Wicker Park,750000,2,1,2000,"Cozy 2-bedroom, 1-bathroom cottage in the ecle...",Wicker Park is a vibrant neighborhood known fo...
6,Gold Coast,1350000,4,4,4500,"Luxurious 4-bedroom, 4-bathroom home in the pr...",Gold Coast is a historic and upscale neighborh...
7,Old Town,850000,3,2,2700,"Classic 3-bedroom, 2-bathroom home in the char...",Old Town is a quaint and picturesque neighborh...
8,South Loop,700000,2,2,2200,"Modern 2-bedroom, 2-bathroom condo in the vibr...",South Loop is a dynamic neighborhood with a mi...
9,River North,1000000,3,3,3000,"Contemporary 3-bedroom, 3-bathroom penthouse i...",River North is a vibrant neighborhood known fo...


In [ ]:
# save the dataframe to a csv file
df.to_csv('real_estate_listings.csv', index_label = 'id')

# Rubrik 2: Semantic Search

##  Rubrik 2.1. : Creating a Vector Database and Storing Listings

In [ ]:
CHROMA_PATH = """croma/"""
CSV_PATH = "real_estate_listings.csv"
embedding_function = OpenAIEmbeddings()

In [ ]:
## create embeddings for each row

df = pd.read_csv(CSV_PATH)
documents = []

df = pd.read_csv(CSV_PATH)
documents = []
for index, row in df.iterrows():
    documents.append(Document(page_content=row['description'], metadata={'id': str(index)}))


# Split Text for each document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

Split 10 documents into 21 chunks.


In [ ]:
if chunks:
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

# Save to Chroma
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

db = Chroma.from_documents(
    chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
)
db.persist()
print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

space is perfect for entertaining, while the private rooftop deck offers stunning city views. Enjoy the convenience of being close to top restaurants, shops, and entertainment venues.
{'id': '4', 'start_index': 199}
Saved 21 chunks to croma/.


## Rubrik 2.2 : Semantic Search of Listings Based on Buyer Preferences

varibla formatted_response contains retrieved context

In [ ]:
query_text = "A comfortable three-bedroom house with a spacious kitchen and a cozy living room along with easy access to Dowtown and have lake view"

BASIC_PROMPT_TEMPLATE =\
"""
Based on the following context:

{context}

---

Answer the question : {question}
"""


In [ ]:
def predict_response(query_text, PROMPT_TEMPLATE):
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
    else:
        context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
        prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        prompt = prompt_template.format(context=context_text, question=query_text)
        print(f"Generated Prompt:\n{prompt}")

        model = ChatOpenAI()
        response_text = model.predict(prompt)
        sources = [doc.metadata.get("id", None) for doc, _score in results]
        formatted_response = f"Response: {response_text}\nSources: {sources}"
        print(formatted_response)

In [ ]:
predict_response(query_text, BASIC_PROMPT_TEMPLATE)

Generated Prompt:
Human: 
Based on the following context:

Charming 2-bedroom, 2-bathroom home in the desirable Lakeview neighborhood. This cozy bungalow features a renovated kitchen, hardwood floors, and a fenced backyard. The spacious living room is perfect for entertaining guests or relaxing with family. Enjoy the convenience of being close to parks,

---

Luxury living in the heart of Downtown. This spacious 4-bedroom, 3-bathroom home features high-end finishes, a gourmet kitchen, and stunning city views. The master suite includes a walk-in closet and spa-like bathroom. Enjoy the convenience of being walking distance to top restaurants, shops, and

---

Classic 3-bedroom, 2-bathroom home in the charming Old Town neighborhood. This historic row house features original architectural details, hardwood floors, and a private patio. The cozy living room is perfect for relaxing by the fireplace or hosting intimate gatherings. Enjoy the convenience of

---

Answer the question : A comforta

# Rubrik 3: Augmented Response Generation

## Rubrik 3.1 and 3.2: Logic for Searching and Augmenting Listing Descriptions

function - predict_response , contains logic for searching and listing augmentation via prompt below

In [ ]:
AUGMENT_PROMPT_TEMPLATE =\
"""
Based on the following context:

{context}

---

craft a response that not only answers the question : {question}, but also ensures that your explanation is distinct, captivating, and customized to align with the specified preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.
"""

In [ ]:
predict_response(query_text, AUGMENT_PROMPT_TEMPLATE)

Generated Prompt:
Human: 
Based on the following context:

Charming 2-bedroom, 2-bathroom home in the desirable Lakeview neighborhood. This cozy bungalow features a renovated kitchen, hardwood floors, and a fenced backyard. The spacious living room is perfect for entertaining guests or relaxing with family. Enjoy the convenience of being close to parks,

---

Luxury living in the heart of Downtown. This spacious 4-bedroom, 3-bathroom home features high-end finishes, a gourmet kitchen, and stunning city views. The master suite includes a walk-in closet and spa-like bathroom. Enjoy the convenience of being walking distance to top restaurants, shops, and

---

Classic 3-bedroom, 2-bathroom home in the charming Old Town neighborhood. This historic row house features original architectural details, hardwood floors, and a private patio. The cozy living room is perfect for relaxing by the fireplace or hosting intimate gatherings. Enjoy the convenience of

---

craft a response that not only a